In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow
import crcmod

#try pip install --user --upgrade scikit-learn==0.20
import sklearn
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

#import dask.dataframe as dd

from datetime import datetime

In [2]:
import keras 

from keras.preprocessing import sequence
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, LeakyReLU
from keras.layers import Embedding
from keras.layers import Concatenate, Flatten, Reshape, Lambda
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, LSTM
from keras.utils import plot_model
from keras import backend as K

from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
#K.tensorflow_backend._get_available_gpus()

In [4]:
#Todo: 
# Correct site 0 values: https://www.kaggle.com/c/ashrae-energy-prediction/discussion/119261
# Leak: https://www.kaggle.com/yamsam/ashrae-leak-data-station
# https://www.kaggle.com/patrick0302/locate-cities-according-weather-temperature
# https://www.kaggle.com/datadugong/locate-better-cities-by-weather-temp-fill-nans
# https://www.kaggle.com/jimeno/ashrae-weather-features-are-not-in-local-time
site_dict ={0:'UCF-Orlando',
           1:'UCL-London',
           2:'ASU-Tempe',
            3:'DC',
           4:'UCB-Berkeley',
            5:'London/Birmingham/Philly',
            6:'DC',
            7:'Ottawa',
            8:'UCF-Orlando',
            9:'San Antonio',
            10:'Salt Lake City',
            11:'Ottawa',
            12:'Dublin',
            13:'Minneapolis',
            14:'Philadelphia',
           15:'Cornell',
           }

#pip install --user feather-format
leak_df = pd.read_csv("gs://inputdata23/leak.csv", index_col=0)
leak_df.head()

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,building_id,meter,meter_reading,timestamp
0,0,0.0,0.0,2016-01-01 00:00:00
1,1,0.0,0.0,2016-01-01 00:00:00
2,2,0.0,0.0,2016-01-01 00:00:00
3,3,0.0,0.0,2016-01-01 00:00:00
4,4,0.0,0.0,2016-01-01 00:00:00


In [5]:
leak_df.fillna(0, inplace=True)
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0
leak_df.timestamp = pd.to_datetime(leak_df.timestamp)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
print (leak_df.duplicated().sum())
#hi


0


Helpful links:
1. Inverse of np.log1p: https://stackoverflow.com/questions/50049891/what-is-the-inverse-of-numpys-log1p
2. Building parallel keras model: https://stackoverflow.com/questions/43151775/how-to-have-parallel-convolutional-layers-in-keras
3. How to make parallel keras models: https://datascience.stackexchange.com/questions/39407/how-to-make-two-parallel-convolutional-neural-networks-in-keras
4. 2 parallel keras layers: https://stackoverflow.com/questions/51546075/two-parallel-conv2d-layers-keras
5. Setting up Conv1D and LSTM: https://stackoverflow.com/questions/51344610/how-to-setup-1d-convolution-and-lstm-in-keras
6. Take average of LSTM hidden states using Lambda: https://stackoverflow.com/questions/51479940/average-channels-of-convolutional-layer-keras
7. Preparing categorical variable for neural networks: https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/
8. Tutorial for using keras for time series (no generators used): https://medium.com/@jdwittenauer/deep-learning-with-keras-structured-time-series-37a66c6aeb28


In [6]:
train = pd.read_csv("gs://inputdata23/train.csv")

def _drop_electrical_zeros(X, y):
    X = X[(y > 0) | (X.meter != 0)]
    y = y.reindex(X.index)
    return X, y

def _drop_missing_site_0(X, y):
    X = X[(X.timestamp >= 3378) | (X.site_id != 0) | (X.meter != 0)]
    y = y.reindex(X.index)
    return X, y

X, y = [(X, y) for (X, y) in train.iterrows()]

train = _drop_electrical_zeros(X, y)

frames = [train, leak_df]
train = pd.concat(frames, sort=True)
train.head()


,building_id,meter,meter_reading,timestamp
0,0,0.0,0.0,2016-01-01 00:00:00
1,1,0.0,0.0,2016-01-01 00:00:00
2,2,0.0,0.0,2016-01-01 00:00:00
3,3,0.0,0.0,2016-01-01 00:00:00
4,4,0.0,0.0,2016-01-01 00:00:00


https://www.kaggle.com/purist1024/strategy-evaluation-what-helps-and-by-how-much/comments
Drop potentially bad data
There are two different optional strategies here that can be used together. _drop_electrical_zeros removes every electrical meter reading with a value of 0; while _drop_missing_site_0 gets rid of all electrical meter readings for site zero during the 141 days that its readings were mostly pegged to 0.

KeyboardInterrupt: 

#### Scaling Meter Reading:
1. nplog1p
2. Group min-max scale

In [ ]:
reading = train['meter_reading']
train['meter_reading'] = np.log1p(train['meter_reading'])

#scaler1 = MinMaxScaler()
#train['meter_reading'] = scaler1.fit_transform(X=np.reshape(train['meter_reading'].values, (-1, 1))).reshape(len(train),)

In [ ]:
train['key'] = train['building_id'].astype(str) + train['meter'].astype(str)


In [ ]:
sc_dict = dict()

def minmaxscaler(x, name, l):
    #print(name)
    key = name
    sc_dict[key]=MinMaxScaler()
    return sc_dict[key].fit_transform(X=np.reshape(x.values, (-1, 1))).reshape(l,)

In [ ]:
train['meter_reading'] = train.groupby('key')['meter_reading'].transform(lambda x: minmaxscaler(x, x.name, len(x)))

In [ ]:
len(sc_dict.keys())

In [ ]:
# for key in train['key'].unique():
#     print(key)
#     l = len(train.loc[train.key == key])
#     scaler1 = MinMaxScaler()
#     train.loc[train.key == key, 'meter_reading'] = scaler1.fit_transform(X=np.reshape(train.loc[train.key == key, 'meter_reading'].values, (-1, 1))).reshape(l,)
#     sc_dict[key] = scaler1

In [ ]:
train['meter_reading'].mean()

In [ ]:
train = train.drop('key', axis=1)

In [ ]:
train.head()

#### Reading Test Data

In [ ]:
test = pd.read_csv("gs://inputdata23/test.csv")

In [ ]:
train.timestamp = pd.to_datetime(train.timestamp)
test.timestamp = pd.to_datetime(test.timestamp)

In [ ]:
len(train), len(test)

#### Reading Weather and Meta Data

In [ ]:
weather_train = pd.read_csv("gs://inputdata23/weather_train.csv")
weather_test = pd.read_csv("gs://inputdata23/weather_test.csv")
meta = pd.read_csv("gs://inputdata23/building_metadata.csv")

In [ ]:
weather_train.timestamp = pd.to_datetime(weather_train.timestamp)
weather_test.timestamp = pd.to_datetime(weather_test.timestamp)

In [ ]:
cols_keep = ['site_id', 'timestamp','air_temperature', 'dew_temperature']#, 'wind_speed']

#### Fetching last hour of weather data

In [ ]:
weather_train = weather_train.sort_values(by = 'timestamp')
weather_test = weather_test.sort_values(by = 'timestamp')

In [ ]:
weather = weather_train.append(weather_test)

In [ ]:
#weather['air_temperature_1'] = weather.groupby(['site_id'])['air_temperature'].shift(1)
#weather['dew_temperature_1'] = weather.groupby(['site_id'])['dew_temperature'].shift(1)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

#weather['air_temperature_2'] = weather.groupby(['site_id'])['air_temperature'].shift(2)
#weather['dew_temperature_2'] = weather.groupby(['site_id'])['dew_temperature'].shift(2)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

#weather['air_temperature_3'] = weather.groupby(['site_id'])['air_temperature'].shift(3)
#weather['dew_temperature_3'] = weather.groupby(['site_id'])['dew_temperature'].shift(3)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

#weather['air_temperature_4'] = weather.groupby(['site_id'])['air_temperature'].shift(4)
#weather['dew_temperature_4'] = weather.groupby(['site_id'])['dew_temperature'].shift(4)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

#weather['air_temperature_5'] = weather.groupby(['site_id'])['air_temperature'].shift(5)
#weather['dew_temperature_5'] = weather.groupby(['site_id'])['dew_temperature'].shift(5)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

#weather['air_temperature_6'] = weather.groupby(['site_id'])['air_temperature'].shift(6)
#weather['dew_temperature_6'] = weather.groupby(['site_id'])['dew_temperature'].shift(6)
#weather['wind_speed_1'] = weather.groupby(['site_id'])['wind_speed'].shift(1)

In [ ]:
cols_keep = cols_keep #+ ['air_temperature_1',# 'dew_temperature_1',
#                         'air_temperature_2',# 'dew_temperature_2',
#                         'air_temperature_3',# 'dew_temperature_3',
#                         'air_temperature_4', #'dew_temperature_4',
#                         'air_temperature_5', #'dew_temperature_5',
#                         'air_temperature_6']#, #'dew_temperature_6']

In [ ]:
train = pd.merge(pd.merge(train, meta, on='building_id', how='left'),
                 weather[cols_keep],
                 on=['site_id', 'timestamp'],
                 how='inner')
test = pd.merge(pd.merge(test, meta, on='building_id', how='left'),
                 weather[cols_keep],
                 on=['site_id', 'timestamp'],
                 how='left')

### site 0 from kBtu to kWh
train.meter_reading[(train.site_id == 0)] = train.meter_reading*0.2931


In [ ]:
train['month'] = train.timestamp.dt.month
train['dayofweek'] = train.timestamp.dt.dayofweek
train['hour'] = train.timestamp.dt.hour

test['month'] = test.timestamp.dt.month
test['dayofweek'] = test.timestamp.dt.dayofweek
test['hour'] = test.timestamp.dt.hour

#### Adding Weekday/Weekend Flag

In [ ]:
train['weekday'] = 0
test['weekday'] = 0
weekday_map = {0:0,
               1:0,
               2:0,
               3:0,
               4:0,
               5:1,
               6:1}
train['weekday'] = train['dayofweek'].map(weekday_map)
test['weekday'] = test['dayofweek'].map(weekday_map)

In [ ]:
test.loc[40525*1024: (40525*1024)+1024]

#### Scaling Data

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
#num_cols

In [ ]:
scaler2 = MinMaxScaler()
num_cols = ['air_temperature', 'dew_temperature'] + ['square_feet', 'floor_count']#+['wind_speed']# + ['air_temperature_1', 
                                                                                                    #'dew_temperature_1',
                                                                                                    #'air_temperature_2', 
                                                                                                    #'dew_temperature_2',
                                                                                                    #'air_temperature_3',
                                                                                                    #'dew_temperature_3',
                                                                                                    #'air_temperature_4', #'dew_temperature_4',
                                                                                                    #'air_temperature_5', #'dew_temperature_5',
                                                                                                    #'air_temperature_6']#, #'dew_temperature_6']
scaler2.fit(train[num_cols])

In [ ]:
train[num_cols] = scaler2.transform(train[num_cols])
test[num_cols] = scaler2.transform(test[num_cols])

#### Joining Weather, Metadata with load data

In [ ]:
len(train), len(test)

In [ ]:
train.columns

In [ ]:
train.head(2)

#### Encoding Categorical Variables

In [ ]:
enc = OrdinalEncoder()
cat_cols = ['site_id', 'hour', 'dayofweek', 'month', 'primary_use']#, 'year_built']
other_cols = ['building_id', 'meter']
enc.fit(train[cat_cols])


In [ ]:
train[cat_cols] = enc.transform(train[cat_cols])
test[cat_cols] = enc.transform(test[cat_cols])

#### Random ID selection

Using 30% of the data as validation data. If more data is needed, we should consider adding data by randomly selecting buildings.

In [ ]:
ids = train['building_id'].unique()

In [ ]:
# keeping 80% of the ids
len_sub = round(len(ids)*0.80)
ids_sub = np.random.choice(ids, len_sub, replace = False)

In [ ]:
# 0.3 represents the percentage of data that is kept for validation
len_val = round(len(ids_sub)*0.3)

In [ ]:
ids_val = np.random.choice(ids_sub, len_val, replace = False)

In [ ]:
ids_train = np.setdiff1d(ids_sub,ids_val)

In [ ]:
assert len(ids_val)+len(ids_train)==len(ids_sub)

#### Applying TimeseriesGenerator to the ASHRAE training data

Once difference between the example above, and our situation is that we have multiple timeseries, for each building and each meter in the building. So, we will be required to modify the code a little bit.

Below, we check how many meters exist in the dataset.

In [ ]:
len(train[['building_id', 'meter']].drop_duplicates()), len(test[['building_id', 'meter']].drop_duplicates())

The code below has been taken from this stackoverflow answer with some modifications:
https://stackoverflow.com/questions/55116638/use-keras-timeseriesgenerator-function-to-generate-squence-group-by-some-id/55118459#55118459

The modification is basically that once we subset the data for building ID, it is then subset for meter type also.

Further reading about modifying keras generator classes can be found below:
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

#### Defining Generators

In [ ]:
num_cols = num_cols

cat_names = cat_cols + other_cols + ['weekday']

#new_cat_names = list(enc.get_feature_names(cat_names))

#col_names = num_cols + new_cat_names
cat_names

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dt, num_cols, cat_names, batch_size = 10):
        self.batch_size = batch_size
        self.len = len(dt)//batch_size
        self.dt = dt[num_cols + cat_names + ['meter_reading']]
        self.num_cols = num_cols
        self.cat_names = cat_names

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index_start = index*self.batch_size
        index_end = index_start+self.batch_size
        adf = self.dt[index_start:index_end]
        return [adf[x] for x in cat_names]+[adf[num_cols]], adf['meter_reading']

In [ ]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, dt, num_cols, cat_names, batch_size = 10):
        self.batch_size = batch_size
        self.len = round(len(dt)/batch_size)
        self.dt = dt[num_cols + cat_names]
        self.num_cols = num_cols
        self.cat_names = cat_names

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index_start = index*self.batch_size
        index_end = index_start+self.batch_size
        adf = self.dt[index_start:index_end]
        return [adf[x] for x in cat_names]+[adf[num_cols]]

#### Training and Validation Generators

In [ ]:
# Test
batch_size = 60*1024#1024+512
train_gen = DataGenerator(train[train['building_id'].isin(ids_train)], 
                          num_cols=num_cols, 
                          cat_names=cat_names,
                          batch_size=batch_size)
val_gen = DataGenerator(train[train['building_id'].isin(ids_val)],
                        num_cols=num_cols, 
                        cat_names=cat_names,
                        batch_size=batch_size)   


In [ ]:
check1 = train_gen[0]
check2 = val_gen[0]

In [ ]:
len(check1[0]), len(check2[0]), len(cat_names)+1

#### Loss Function - Root Mean Square

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

#### Embeddings NN

1. https://towardsdatascience.com/decoded-entity-embeddings-of-categorical-variables-in-neural-networks-1d2468311635
2. https://medium.com/@satnalikamayank12/on-learning-embeddings-for-categorical-data-using-keras-165ff2773fc9
3. https://github.com/mayanksatnalika/ipython/blob/master/embeddings%20project/cycle_sharing/entity_embeddings_regression.ipynb

In [ ]:
train.columns

In [ ]:
len(train.columns)

In [ ]:
len(cat_names), len(num_cols)

In [ ]:
models = []
inputs = []
act = 'relu'
n_num_cols = len(num_cols)


for categorical_var in cat_names:
    model = Sequential()
    model.reset_states()
    no_of_unique_cat  = train[categorical_var].nunique()
    embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
    embedding_size = int(embedding_size)
    input1 = Input(shape=(1,))
    model = Embedding(no_of_unique_cat+1,embedding_size)(input1)
    model = Reshape(target_shape=(embedding_size,))(model)
    models.append(model)
    inputs.append(input1)
    
input1 = Input(shape=(n_num_cols,))
model_rest = Dense(100, activation=act)(input1)
models.append(model_rest)
inputs.append(input1)

m1 = keras.layers.concatenate(models, axis = 1)
m1 = Dropout(0.5)(m1)
m1 = Dense(200, activation=act)(m1)
m1 = Dropout(0.5)(m1)
m1 = Dense(5, activation=act)(m1)
m1 = LeakyReLU(alpha=0.1)(m1)
m1 = Dropout(0.2)(m1)
m1 = Dense(1, activation='sigmoid')(m1)

model3 = Model(inputs = inputs, outputs = m1)

opt = keras.optimizers.RMSprop(clipnorm=1.)

model3.compile(loss=root_mean_squared_error, optimizer=opt, metrics=['mse', 'mae', 'mape'])

In [ ]:
plot_model(model3)

In [ ]:
#model3.summary()

In [ ]:
epochs = 10#TODO: change it back to 5
workers = 10
model3.fit_generator(generator=train_gen,
                    validation_data=val_gen, epochs=epochs, 
                    use_multiprocessing=True, workers = workers)

In [ ]:
# model_rest with Dense(65), epoch 1: loss=0.2594, val_loss=0.2829, val_loss somewhat constant
# model_rest with Dense(120), epoch 1: loss=0.2589, val_loss=0.2835, val_loss somewhat constant
# model_rest with Dense(300), epoch 1: loss=0.2594, val_loss=0.2830, val_loss somewhat constant
# model_rest with Dense(65), combined model with Dense(200), epoch 1: loss=0.2554, val_loss=0.2858, val_loss reduces to 0.2550 by epoch 5

1. Dense(64) for model_rest, epoch 1: loss=0.0656, val_loss=0.1382, val_loss increased
2. Replacing the Dense(64) for model_rest with Dense(24), epoch 1: loss: 0.0648, val_loss: 0.1481, val_loss increased

In [ ]:
val_test_gen = TestDataGenerator(train[train['building_id'].isin(ids_val[0:20])], 
                                 num_cols=num_cols, 
                                cat_names=cat_names,
                                batch_size=batch_size)
print(len(val_test_gen))

val_test_res = model3.predict_generator(generator=val_test_gen, workers=12, use_multiprocessing=True)

x = []
for i in range(len(val_test_gen)):
    x.append(val_gen[i][1].values)


In [ ]:
dummy = train[train['building_id'].isin(ids_val[0:20])].copy()


In [ ]:
dummy['y'] = val_test_res
dummy['key'] = dummy['building_id'].astype(str) + dummy['meter'].astype(str)


In [ ]:
def inverseminmax(x, name, l):
    key = name
    return sc_dict[key].inverse_transform(np.reshape(x.values, (-1, 1))).reshape(l,)

In [ ]:
val_test_y = np.expm1(dummy.groupby('key')['y'].transform(lambda x: inverseminmax(x, x.name, len(x))))

In [ ]:
#print(len(val_test_y) == len(val_test_res_og))

In [ ]:
print("Error: " +str({np.sqrt(np.mean(np.square(val_test_res_og - val_test_y)))}))

#### Defining Function for Prediction - Method 1

In [ ]:
test_gen = TestDataGenerator(test, num_cols=num_cols, 
                        cat_names=cat_names,
                        batch_size=batch_size)


In [ ]:
len(test_gen)

In [ ]:
cat_cols

In [ ]:
#enc.categories_

In [ ]:
#test.loc[40525*1024: (40525*1024)+1024]#.dt.month

In [ ]:
result = model3.predict_generator(generator=test_gen, workers=12, use_multiprocessing=True, verbose=1)
result.head()

In [ ]:
result# = None

In [ ]:
result.shape

In [ ]:
#pd.Series(np.expm1(scaler1.inverse_transform(result)).reshape(result.shape[0],))

In [ ]:
#len(pd.read_csv("gs://123test_bucket/test.csv"))

In [ ]:
41697600/batch_size

In [ ]:
(test.loc[0:10,'building_id'].astype(str) + test.loc[0:10,'meter'].astype(str)).values

In [ ]:
result_df = pd.DataFrame()
result_df['key'] = (test['building_id'].astype(str) + test['meter'].astype(str)).values

In [ ]:
result_df['meter_reading']=result
result_df['row_id']=test['row_id']

In [ ]:
result_df.head()

In [ ]:
result_df['meter_reading'] = np.expm1(result_df.groupby('key')['meter_reading'].transform(lambda x: inverseminmax(x, x.name, len(x))))

In [ ]:
result_df = result_df[['row_id', 'meter_reading']]

In [ ]:
result_df.head(2)

In [ ]:
result_df.shape

In [ ]:
result_df.to_csv("gs://inputdata23/result.csv", index=False)

In [89]:
check = pd.read_csv("result.csv")
check.shape

(2380, 6)

In [90]:
result_df[result_df.meter_reading.isnull()]

,row_id,meter_reading


In [74]:
sample = pd.read_csv("gs://123test_bucket/sample_submission.csv")

In [76]:
sample['meter_reading'] = result_df['meter_reading']

In [77]:
sample = sample.fillna(0)

In [78]:
sample.to_csv("result_sample.csv", index=False)

In [79]:
# serialize model to JSON
model_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model2.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
